# São Paulo 07 de Agosto 2019
## Competition Description

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.
One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.
In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

### Arquivos: {train.csv, test.csv, gender_submission.csv}

In [1]:
#Instalando xgboot
!pip install --upgrade pip
!pip install xgboost

     |████████████████████████████████| 1.4MB 2.8MB/s eta 0:00:01
  Found existing installation: pip 19.1.1
    Uninstalling pip-19.1.1:
      Successfully uninstalled pip-19.1.1
     |████████████████████████████████| 142.8MB 6.7kB/s  eta 0:00:01    |██████                          | 26.4MB 6.3MB/s eta 0:00:19     |██████▊                         | 30.1MB 6.3MB/s eta 0:00:18MB/s eta 0:00:18��█████▍                      | 41.8MB 6.5MB/s eta 0:00:16        | 57.2MB 13.2MB/s eta 0:00:07��█████████▋                  | 60.9MB 13.2MB/s eta 0:00:07MB 602kB/s eta 0:02:05            | 69.5MB 602kB/s eta 0:02:02    | 76.8MB 14.1MB/s eta 0:00:05    | 79.5MB 14.1MB/s eta 0:00:05    | 80.3MB 14.1MB/s eta 0:00:054.1MB/s eta 0:00:05��██▊             | 83.5MB 4.6MB/s eta 0:00:14��███             | 85.2MB 4.6MB/s eta 0:00:13��███████████████▎            | 86.1MB 4.6MB/s eta 0:00:13��███████████████▋            | 87.4MB 4.6MB/s eta 0:00:13��████████████████            | 89.0MB 4.6MB/s eta 0:00:12    | 

In [2]:
# Realizando os imports
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [3]:
# Load the data
train_df = pd.read_csv('data/train.csv', header=0)
test_df = pd.read_csv('data/test.csv', header=0)


In [4]:
# We'll impute missing values using the median for numeric columns and the most
# common value for string columns.
# This is based on some nice code by 'sveitser' at http://stackoverflow.com/a/25562948
from sklearn.base import TransformerMixin

In [5]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Muitos valores aqui são inuteis tais como Name, PassengerId e Embarked.
train_df = train_df[['Survived', 'Pclass', 'Sex', 'Age']]

In [7]:
#Alguns valores estão faltando na Idade
train_df.count()

Survived    891
Pclass      891
Sex         891
Age         714
dtype: int64

In [8]:
# São 177
train_df['Age'].isnull().sum()

177

In [9]:
# A media das idades maiores que 0 é de 30 anos
train_df.loc[train_df['Age'] > 0].mean()

Survived     0.406162
Pclass       2.236695
Age         29.699118
dtype: float64

In [10]:
# Adcionando o valor idade media na idade geral
train_df.loc[train_df.Age.isnull() == True, 'Age'] = 30

In [11]:
train_df.head(7)

,Survived,Pclass,Sex,Age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0
5,0,3,male,30.0
6,0,1,male,54.0


In [12]:
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

feature_columns_to_use = ['Pclass','Sex','Age']
nonnumeric_columns = ['Sex']

In [13]:
# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)

In [14]:
# XGBoost doesn't (yet) handle categorical features automatically, so we need to change
# them to columns of integer values.
# See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more
# details and options
le = LabelEncoder()
for feature in nonnumeric_columns: 
    big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

In [15]:
# Prepare the inputs for the model
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['Survived']



In [16]:
# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, train_y)
predictions = gbm.predict(test_X)


/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test_df['PassengerId'],
                            'Survived': predictions })
submission.to_csv("data/submission10.csv", index=False)